In [1]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
import src.job_scrape_funcs as scrape
import src.analysis as ana
from datetime import datetime
from pathlib import Path

c:\Users\tempu\Documents\Code\job_tools\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def job_pipeline(job_title, location, post_time, pages, save_path, resume_path):
    #TODO: try except statements
    # Scrape jobs:
    job_data = scrape.scrape_jobs(pages=pages, job_title=job_title, location=location, post_time=post_time)
    date = datetime.today().strftime("%Y-%m-%d")

    # Save:
    index = 1
    full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"
    while (Path.cwd()/full_path).exists():
        index += 1
        full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"

    print("Saving to ", full_path)
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(job_data, f, ensure_ascii=False, indent=4)

    # Find matches:
    with open(full_path, 'r', encoding="utf-8") as f:
        data = json.load(f)

    raw_df = pd.DataFrame(data["jobs"])
    df = ana.find_job_match(raw_df, resume_path)
    print("Generating report:\n\n")
    ana.generate_report(df)
    return df

In [ ]:
# Scrape job listings:
jobs = scrape.scrape_jobs(pages=2, job_title="Data Scientist", location="Chicago", post_time=5)

In [8]:
# Save to JSON:
with open("data/job_queries/job_data_test.json", 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

In [8]:
# Load JSON:
with open("data/job_queries/Data-Scientist_2025-04-08.json", 'r', encoding='utf-8') as f:
    jobs = json.load(f)
raw_df = pd.DataFrame(jobs["jobs"])

In [ ]:
# Find job matches:
df = ana.find_job_match(raw_df, "data/resume.pdf")
df.head()

In [3]:
df = job_pipeline(job_title="Research Scientist", 
                  location="Chicago", 
                  post_time=7, 
                  pages=10,
                  save_path="data/job_queries/", 
                  resume_path="resume.pdf")

Scraping page 0
Failed to retrieve job details: 429
10 scraped successfully.
Scraping page 1
20 scraped successfully.
Scraping page 2
Failed to retrieve job details: 429
30 scraped successfully.
Scraping page 3
40 scraped successfully.
Scraping page 4
Failed to retrieve job details: 429
50 scraped successfully.
Scraping page 5
60 scraped successfully.
Scraping page 6
70 scraped successfully.
Scraping page 7
71 scraped successfully.
Scraping page 8
No more jobs found.
Exception occured during scraping.
'NoneType' object is not iterable
Saving to  data/job_queries/Research-Scientist_2025-04-08_1.json
Generating report:


Staff ML Scientist, AI Research & Development
Tempus AI
Chicago, IL
https://www.linkedin.com/jobs/view/staff-ml-scientist-ai-research-development-at-tempus-ai-4199931480?position=5&pageNum=3&refId=Kn3hbcvH7OqIEtpoAUGxrg%3D%3D&trackingId=2aGNHWrSDAoN9YYZuWe9BA%3D%3D
Matching keywords: {'models', 'fusion', 'Innovation', 'Research', 'modeling', 'approach', 'journals', 'lang

In [ ]:
# generate job match report
ana.generate_report(df)

Lead AI/ML Engineer (P2122)
84.51˚
Chicago, IL
https://www.linkedin.com/jobs/view/lead-ai-ml-engineer-p2122-at-84-51%CB%9A-4099652285?position=3&pageNum=7&refId=cYbFyg8baLtUpc4i%2F3NU%2Fg%3D%3D&trackingId=V6lLW1J4EfyzZsd%2FezBQqg%3D%3D
Matching keywords: {'ML', 'time', 'matching', 'data', 'models', 'art', 'pipeline', 'model', 'Engineering', 'contribution', 'control', 'Science', 'machine', 'PyTorch', 'Research', 'feedback', 'members', 'learning', 'k', 'methods', 'research', 'TensorFlow', 'test', 'scientist', 'state', 'Python', 'Medical'}
Description:
	 Technical ML Engineer lead on larger projects that can span months
	Foster a collaborative and innovative team environment, encouraging professional growth and development among junior team members.
	Leverage known patterns, frameworks, and tools for automating & deploying machine learning solutions
	Develop new tools, processes and operational capabilities to monitor and analyze model performance and data accuracy where needed
	Work with